In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Reads in csv file made in WeatherPy program
citydata = pd.read_csv("../WeatherPy/Output_Data/cities.csv")

# Stores the latitude and longitude for location and humidity as the "weight" for heatmap
locations = citydata[["Lat", "Lng"]]
rating = citydata["Humidity"]

# Creates the fig to display the heatmap
fig = gmaps.figure()

# Creates heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
# Add layer and display figure
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Narrowing down cities by condition until there's 10
focusedcitydata = citydata.loc[citydata["Max Temp"] > 70]
focusedcitydata = focusedcitydata.loc[focusedcitydata["Max Temp"] < 75]
focusedcitydata = focusedcitydata.loc[focusedcitydata["Humidity"] < 50]
focusedcitydata = focusedcitydata.loc[focusedcitydata["Wind Speed"] < 12]

# Removing unnecessary rows from the data frame and adding hotel name column
focusedcitydata = focusedcitydata[["City", "Lat", "Lng", "Country"]]
focusedcitydata["Hotel Name"] = ""
hotel_df = focusedcitydata
hotel_df

,City,Lat,Lng,Country,Hotel Name
31,Mount Isa,-20.7333,139.5000,AU,
79,Saint George,37.1041,-113.5841,US,
435,Shache,38.4167,77.2406,CN,
440,Buin,-33.7333,-70.7500,CL,
485,Tabas,33.5959,56.9244,IR,
523,Marzuq,14.4000,46.4667,YE,


In [17]:
# Base URL for the nearby search google API
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting up base parameters for the API request
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key  
}

# Loops through the data frame
for index, row in hotel_df.iterrows():
    
    # Stores the latitude and longitude for each city
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the location to the parameters for API request
    params["location"] = f"{lat}, {lng}"
    
    # Makes the API request
    hotelsearch = requests.get(url, params=params).json()
    
    # Try-Except in case a hotel can't be found the program doesn't break
    try:
        hotel_df.loc[index, "Hotel Name"] = hotelsearch["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Missing result... skipping")

hotel_df

,City,Lat,Lng,Country,Hotel Name
31,Mount Isa,-20.7333,139.5000,AU,Mount Isa
79,Saint George,37.1041,-113.5841,US,St. George
435,Shache,38.4167,77.2406,CN,Kashgar Prefecture
440,Buin,-33.7333,-70.7500,CL,San Bernardo
485,Tabas,33.5959,56.9244,IR,Tabas
523,Marzuq,14.4000,46.4667,YE,Al Hujayrah


In [24]:
# Sets up the hotel markers for the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row into the marker template
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Stores the latitude and longitude for each hotel
locations = hotel_df[["Lat", "Lng"]]

# Adds markers to this layer for each hotel
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add layer and display figure
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))